In [1]:
import spacy
import json
import os
from pathlib import Path
import random
from spacy.training import Example
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from spacy.training import offsets_to_biluo_tags, biluo_tags_to_offsets

In [12]:
BASE_DIR = Path.cwd().resolve().parents[2]
DATA_DIR = os.path.join(BASE_DIR, 'data', 'cleaned_standard_annotated')
FOLDER_NAMES = ['Python', 'ML', 'Android', 'DevOps', 'dotNET', 'FrontEnd', 'Golang', 'Java', 'macOS', 'Node', 'PHP']
TAGS_PATH = os.path.join(BASE_DIR, 'data', 'annotated_manual', 'tags.json')
SAVE_PATH = 'spacy_model'

In [3]:
# Loading annotated NER data
def load_annotated_data(base_dir):
    data = []
    for folder in FOLDER_NAMES:
        folder_path = os.path.join(DATA_DIR, folder)
        if not os.path.exists(folder_path):
            continue

        for root, _, files in os.walk(folder_path):
            for filename in files:
                if filename.endswith(".json"):
                    file_path = os.path.join(root, filename)
                    with open(file_path, "r", encoding="utf-8") as f:
                        file = json.load(f)
                        text = file['text']
                        entities = [
                            (start, end, label)
                            for start, end, label in file.get("entities", [])
                        ]
                        if text.strip():
                            data.append((text, {"entities": entities}))

    print(f"Loaded {len(data)} annotated samples")
    return data

data = load_annotated_data(DATA_DIR)

Loaded 702 annotated samples


In [4]:
with open(TAGS_PATH, "r", encoding="utf-8") as f:
    tags = json.load(f)

labels = [t["name"] for t in tags]
print(f"Loaded {len(labels)} entity labels from tags.json:\n{labels}")

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
print(f"Train: {len(train_data)} | Test: {len(test_data)}")

nlp = spacy.blank("en") 
for pipe_name in nlp.pipe_names:
    nlp.remove_pipe(pipe_name)

ner = nlp.add_pipe("ner")

for label in labels:
    ner.add_label(label)

Loaded 10 entity labels from tags.json:
['SKILL_HARD', 'SKILL_SOFT', 'ENGLISH_LEVEL', 'DEGREE', 'EXPERIENCE_LEVEL', 'EXPERIENCE_YEARS', 'BENEFIT', 'LOCATION', 'COMPANY_NAME', 'ROLE']
Train: 561 | Test: 141


In [ ]:
def remove_overlapping_entities(entities):
    entities = sorted(entities, key=lambda x: (x[0], x[1]))
    cleaned = []
    last_end = -1

    for start, end, label in entities:
        if start < last_end:   # Overlap → skip
            continue
        cleaned.append((start, end, label))
        last_end = end

    return cleaned

cleaned_train = []
for text, ann in train_data:
    ann = ann.copy()
    ann["entities"] = remove_overlapping_entities(ann["entities"])
    cleaned_train.append((text, ann))

cleaned_test = []
for text, ann in test_data:
    ann = ann.copy()
    ann["entities"] = remove_overlapping_entities(ann["entities"])
    cleaned_test.append((text, ann))

train_data = cleaned_train
test_data = cleaned_test

optimizer = nlp.initialize()
n_iter = 100

for i in range(n_iter):
    random.shuffle(train_data)
    losses = {}

    for text, annotations in tqdm(train_data, desc=f"Epoch {i+1}/{n_iter}"):

        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)

        nlp.update([example], drop=0.2, sgd=optimizer, losses=losses)

    print(f"Losses at iteration {i+1}: {losses}")


correct, total = 0, 0

for text, ann in test_data:
    doc = nlp(text)

    pred = {(e.start_char, e.end_char, e.label_) for e in doc.ents}
    true = set(tuple(e) for e in ann["entities"])

    correct += len(pred & true)
    total += len(true)

print("Accuracy:", correct / total if total > 0 else 0)


Epoch 1/100:   1%|          | 6/561 [00:00<01:13,  7.50it/s]c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Senior Full-Stack Developer (Jakarta EE+Angular)
O..." with entities "[(0, 6, 'EXPERIENCE_LEVEL'), (29, 39, 'SKILL_HARD'...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
Epoch 1/100:   2%|▏         | 10/561 [00:01<01:01,  8.90it/s]c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Senior Full Stack (.NET+React) Engineer
About the ..." with entities "[(0, 6, 'EXPERIENCE_LEVEL'), (24, 29, 'SKILL_HARD'...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligne

Losses at iteration 1: {'ner': 21766.34853847169}


Epoch 2/100: 100%|██████████| 561/561 [01:05<00:00,  8.57it/s]


Losses at iteration 2: {'ner': 12117.310899817357}


Epoch 3/100: 100%|██████████| 561/561 [01:05<00:00,  8.58it/s]


...

Losses at iteration 98: {'ner': 2127.8284146729584}


Epoch 99/100: 100%|██████████| 561/561 [01:05<00:00,  8.55it/s]


Losses at iteration 99: {'ner': 2061.8562787310175}


Epoch 100/100: 100%|██████████| 561/561 [01:08<00:00,  8.25it/s]


Losses at iteration 100: {'ner': 2113.113573886627}
Accuracy: 0.8222958057395143


In [9]:
precision = correct / total if total > 0 else 0
print(f"Baseline precision: {precision:.3f}")

Baseline precision: 0.822


In [10]:
example_text = random.choice(test_data)[0]
doc = nlp(example_text)
print("Example predictions:")
for ent in doc.ents:
    print(f"{ent.text:<30} → {ent.label_}")

Example predictions:
Team Lead                      → ROLE
Україні                        → LOCATION
5+ years                       → EXPERIENCE_YEARS
OOP                            → SKILL_HARD
SOLID                          → SKILL_HARD
Android Architecture components → SKILL_HARD
Android UI design principles   → SKILL_HARD
Android SDK                    → SKILL_HARD
Kotlin                         → SKILL_HARD
Client-Server architecture     → SKILL_HARD
Rest
API                       → SKILL_HARD
SQL                            → SKILL_HARD
Clean Architecture             → SKILL_HARD
MVVM Pattern                   → SKILL_HARD
jetpack                        → SKILL_HARD
medical insurance              → BENEFIT
flexible schedule              → BENEFIT
paid vacation                  → BENEFIT
sick days                      → BENEFIT
Odessa                         → LOCATION
Nayax                          → COMPANY_NAME
Laundromats                    → SKILL_HARD
Nayax                   

In [11]:
examples = []
for text, annotations in test_data:
    doc = nlp.make_doc(text)
    examples.append(Example.from_dict(doc, annotations))

scorer = nlp.evaluate(examples)

print("Evaluation results:")
print(f"Precision: {scorer['ents_p']:.3f}")
print(f"Recall:    {scorer['ents_r']:.3f}")
print(f"F-score:   {scorer['ents_f']:.3f}")
print(f"Entities per type:\n{scorer['ents_per_type']}")

c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Team Lead, Full Stack Engineering
At SPD Technolog..." with entities "[(0, 9, 'ROLE'), (11, 33, 'ROLE'), (37, 51, 'COMPA...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "BackEnd (Node.js) Developer
Moon Active is one of ..." with entities "[(9, 16, 'SKILL_HARD'), (28, 39, 'COMPANY_NAME'), ...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\Admin\AppData\Local\Programs\Python\Python310

Evaluation results:
Precision: 0.830
Recall:    0.829
F-score:   0.830
Entities per type:
{'EXPERIENCE_LEVEL': {'p': 0.8716577540106952, 'r': 0.9157303370786517, 'f': 0.893150684931507}, 'COMPANY_NAME': {'p': 0.842443729903537, 'r': 0.6986666666666667, 'f': 0.7638483965014579}, 'SKILL_HARD': {'p': 0.8370642201834863, 'r': 0.860754716981132, 'f': 0.8487441860465116}, 'LOCATION': {'p': 0.9041095890410958, 'r': 0.8275862068965517, 'f': 0.8641571194762684}, 'ROLE': {'p': 0.7483443708609272, 'r': 0.601063829787234, 'f': 0.6666666666666667}, 'EXPERIENCE_YEARS': {'p': 0.9577464788732394, 'r': 0.9444444444444444, 'f': 0.951048951048951}, 'SKILL_SOFT': {'p': 0.6739130434782609, 'r': 0.7524271844660194, 'f': 0.7110091743119266}, 'ENGLISH_LEVEL': {'p': 0.7971014492753623, 'r': 0.7857142857142857, 'f': 0.7913669064748201}, 'BENEFIT': {'p': 0.782016348773842, 'r': 0.82, 'f': 0.800557880055788}, 'DEGREE': {'p': 0.6923076923076923, 'r': 0.6428571428571429, 'f': 0.6666666666666666}}


In [ ]:
os.makedirs(SAVE_PATH, exist_ok=True)
nlp.to_disk(SAVE_PATH)